In [0]:

print(spark)

In [0]:
#Order by & Sort
simpleData = [("James","Sales","NY",90000,34,10000), \
    ("Michael","Sales","NY",86000,56,20000), \
    ("Robert","Sales","CA",81000,30,23000), \
    ("Maria","Finance","CA",90000,24,23000), \
    ("Raman","Finance","CA",99000,40,24000), \
    ("Scott","Finance","NY",83000,36,19000), \
    ("Jen","Finance","NY",79000,53,15000), \
    ("Jeff","Marketing","CA",80000,25,18000), \
    ("Kumar","Marketing","NY",91000,50,21000) \
  ]
columns= ["employee_name","department","state","salary","age","bonus"]
 
df = spark.createDataFrame(data = simpleData, schema = columns)

df.sort(df.department.asc(),df.state.asc()).show(truncate=False)
df.orderBy(df.department.asc(),df.state.asc()).show(truncate=False)

In [0]:
#Union
simpleData = [("James","Sales","NY",90000,34,10000), 
    ("Michael","Sales","NY",86000,56,20000), \
    ("Robert","Sales","CA",81000,30,23000), \
    ("Maria","Finance","CA",90000,24,23000) \
  ]
 
columns= ["employee_name","department","state","salary","age","bonus"]
df = spark.createDataFrame(data = simpleData, schema = columns)
 
simpleData2 = [("James","Sales","NY",90000,34,10000), \
    ("Maria","Finance","CA",90000,24,23000), \
    ("Jen","Finance","NY",79000,53,15000), \
    ("Jeff","Marketing","CA",80000,25,18000), \
    ("Kumar","Marketing","NY",91000,50,21000) \
  ]
columns2= ["employee_name","department","state","salary","age","bonus"]

df2 = spark.createDataFrame(data = simpleData2, schema = columns2)
 
union_df = df.union(df2).distinct()
union_df.show(truncate=False)

In [0]:
#UnionByName

simpleData = [("James","Sales","NY",90000,34,10000), 
    ("Michael","Sales","NY",86000,56,20000), \
    ("Robert","Sales","CA",81000,30,23000), \
    ("Maria","Finance","CA",90000,24,23000) \
  ]
 
columns= ["employee_name","department","state","salary","age","bonus"]
df = spark.createDataFrame(data = simpleData, schema = columns)
 
simpleData2 = [("James","Sales","NY",90000,34,10000), \
    ("Maria","Finance","CA",90000,24,23000), \
    ("Jen","Finance","NY",79000,53,15000), \
    ("Jeff","Marketing","CA",80000,25,18000), \
    ("Kumar","Marketing","NY",91000,50,21000) \
  ]
columns2= ["employee_name","department","state","salary","age","bonus"]

df2 = spark.createDataFrame(data = simpleData2, schema = columns2)

from pyspark.sql.functions import col

unionbyname_df = df.unionByName(df2).distinct().orderBy(col("employee_name").asc())
unionbyname_df.show(truncate=False)

In [0]:
#Joins - inner, left, right, leftanti, leftsemi, cross

data_employees = [
    (1, "James", "Sales", "NY"),
    (2, "Michael", "Sales", "NY"),
    (3, "Robert", "Sales", "CA"),
    (4, "Maria", "Finance", "CA"),
    (5, "Jen", "Finance", "NY"),
    (6, "Jeff", "Marketing", "CA"),
    (7, "Kumar", "Marketing", "NY")
]
columns_employees = ["emp_id", "employee_name", "department", "state"]
employees_df = spark.createDataFrame(data_employees, columns_employees)

data_departments = [
    ("Sales", "John"),
    ("Finance", "Samantha"),
    ("Marketing", "Steve"),
    ("IT", "Nancy")
]
columns_departments = ["department", "manager"]
departments_df = spark.createDataFrame(data_departments, columns_departments)

data_salaries = [
    (1, 90000, 10000),
    (2, 86000, 20000),
    (3, 81000, 23000),
    (4, 90000, 23000),
    (5, 79000, 15000),
    (6, 80000, 18000),
    (7, 91000, 21000)
]
columns_salaries = ["emp_id", "salary", "bonus"]
salaries_df = spark.createDataFrame(data_salaries, columns_salaries)

data_states = [
    ("NY", "New York"),
    ("CA", "California"),
    ("TX", "Texas")
]
columns_states = ["state", "state_name"]
states_df = spark.createDataFrame(data_states, columns_states)


emp_dept_df = employees_df.join(departments_df, employees_df.department == departments_df.department, "inner")
emp_dept_sal_df = emp_dept_df.join(salaries_df, emp_dept_df.emp_id == salaries_df.emp_id, "inner")
emp_dept_sal_state_df = emp_dept_sal_df.join(states_df, emp_dept_sal_df.state == states_df.state, "inner")

#display(emp_dept_sal_state_df)

final_df = emp_dept_sal_state_df.select(employees_df.employee_name, departments_df.department,departments_df.manager, states_df.state, salaries_df.salary, salaries_df.bonus).orderBy(employees_df.employee_name)

display(final_df)

In [0]:
# for loop
sample_data = [("Alice", 28, "Engineer"),
               ("Bob", 34, "Doctor"),
               ("Cathy", 29, "Teacher"),
               ("David", 42, "Artist")]

columns = ["name","age","profession"]

df = spark.createDataFrame(data = sample_data, schema= columns)

for i in df.collect():
    print(i.name, i.age, i.profession )

In [0]:
import time
from pyspark.sql import functions as F
 
 
# Create largefrom pyspark.sql import functions as F
#dataset
df = spark.range(0, 10_000_000)
df_transformed = df.withColumn("square", df["id"] * df["id"])
# First action: count()
start = time.time()
df_transformed.count()
print("Without Cache - First count done in:", time.time() - start, "seconds")
 
 
 
# Second action: sum()
start = time.time()
df_transformed.agg(F.sum("square")).show()
print("Without Cache - Second aggregation done in:", time.time() - start, "seconds")
 
 
 
# Create large dataset
df = spark.range(0, 10_000_000)
df_transformed = df.withColumn("square", df["id"] * df["id"])
# Apply caching
df_transformed.cache()
# First action: count() triggers caching
start = time.time()
df_transformed.count() #first action is slower (because it's doing both compute + caching).
print("With Cache - First count done in:", time.time() - start, "seconds")
 
# Second action: sum() uses cached data
start = time.time()
df_transformed.agg(F.sum("square")).show()
print("With Cache - Second aggregation done in:", time.time() - start, "seconds")

In [0]:
def convertCase(str):
    resStr=""
    arr = str.split(" ")
    for x in arr:
       resStr= resStr + x[0:1].upper() + x[1:len(x)] + " "
    return resStr

convertCase("anil kumar")

In [0]:
spark.sql("SHOW TABLES IN global_temp").show()
 
spark.catalog.listTables()  # Lists tables and views in the current database
spark.sql("SHOW TABLES").show()


In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import when

data = [
    Row(id=1, name="Alice", age=23, salary=5000, dept="HR"),
    Row(id=2, name="Bob", age=35, salary=7000, dept="IT"),
    Row(id=3, name="Charlie", age=30, salary=4000, dept="HR"),
    Row(id=4, name="David", age=45, salary=9000, dept="Finance"),
    Row(id=5, name="Eve", age=28, salary=6000, dept="IT"),
    Row(id=6, name="Frank", age=50, salary=10000, dept="Finance"),
    Row(id=7, name="Grace", age=27, salary=3000, dept="HR"),
    Row(id=8, name="Heidi", age=32, salary=8000, dept="IT"),
    Row(id=9, name="Ivan", age=41, salary=9500, dept="Finance"),
    Row(id=10, name="Judy", age=29, salary=5500, dept="HR")
]

df = spark.createDataFrame(data)
#display(df)

case_df = df.withColumn("Salary_category", when (col("salary") > 5000, "High")
                                            .when ((col("salary") > 5000) & (col("salary") < 8000), "Medium" ).otherwise("Low"))
display(case_df)

In [0]:
# Sample data with nulls
data = [("Alice", 25), ("Bob", None), ("Charlie", 30), (None, 40)]
columns = ["name", "age"]

df = spark.createDataFrame(data, columns)
#display(df)

# Replace nulls with a default value
from pyspark.sql.functions import when

case_df = df.withColumn("age", when(col("age").isNull(), 89)
                        .when(col("age")>50, 60)
                        .otherwise(col("age")))
display(case_df)